# **Naver News Search Open API**
### **목표**
1. **네이버 뉴스 검색 API 호출**:  
   - 키워드를 기반으로 뉴스 데이터를 검색합니다.
2. **데이터 분석 및 활용**:  
   - 검색된 데이터를 정리 및 가공하여 원하는 정보를 추출합니다.
3. **뉴스 데이터 웹크롤링**:  
   - API로 제공되지 않는 추가 데이터를 수집하기 위해 웹 크롤링을 수행합니다.

### **작성자 정보**
Written by [**Yujin_nKim**](https://github.com/Yujin-nKim)

In [38]:
# 네이버 API 인증 키 설정 
# 네이버 개발자 센터에서 발급받은 클라이언트 ID와 시크릿 키를 이용합니다. 
my_client_id = "FHNP6NJ_GuKgxhnCOjUi" # 네이버 클라이언트 ID
my_client_secret = "kgX6c9lqKR" # 네이버 클라이언트 시크릿 키 

# 네이버 뉴스 검색 APi의 엔드포인트 URL (json 형식)
base_url = "https://openapi.naver.com/v1/search/news.json";

In [39]:
# 검색 파라미터 설정
searchWord = "재테크" # 검색어
display = 100 # 한 번에 표시할 검색 결과 개수 (기본값: 10, 최댓값: 100)
start = 1 # 검색 시작 위치 (기본값: 1, 최댓값: 1000)
sort = "sim" # 검색 결과 정렬 방식 (sim - 정확도순 (기본값, 내림차순) / date - 날짜순 (내림차순)))

In [40]:
# API 호출 결과로 받은 뉴스 데이터 리스트를 출력하는 함수 
# items (list) : 뉴스 검색 결과의 아이템 목록
def print_item(items):
    for item in items:
        print("제목:", item["title"].replace("<b>", "").replace("</b>", ""))
        print("요약:", item["description"].replace("<b>", "").replace("</b>", ""))
        print("링크:", item["originallink"])
        print("작성일:", item["pubDate"])
        print("-" * 50)

In [41]:
import csv

# 뉴스 검색 결과를 CSV 파일로 저장하는 함수
# items (list) : 뉴스 검색 결과의 아이템 목록
# filename (str) : 저장할 CSV 파일 이름 (기본값: 'news_results.csv')
def save_to_csv(items, filename="news_results.csv"):
    # CSV 파일 열기
    with open(filename, mode="w", encoding="utf-8", newline="") as file:
        writer = csv.writer(file)
        
        # 헤더 작성
        writer.writerow(["제목", "요약", "원문 링크", "네이버 뉴스 링크", "작성일"])
        
        # 데이터 작성
        for item in items:
            title = item["title"].replace("<b>", "").replace("</b>", "")
            description = item["description"].replace("<b>", "").replace("</b>", "")
            originallink = item["originallink"]
            link = item["link"]
            pubDate = item["pubDate"]
            
            writer.writerow([title, description, originallink, link, pubDate])

    print(f"결과가 {filename}에 저장되었습니다.")

In [42]:
# 네이버 검색 API 예제 - 뉴스 검색

import urllib.request
import json

# 네이버 API 인증 정보
client_id = my_client_id 
client_secret = my_client_secret

all_items = [] # 모든 뉴스 데이터를 저장할 리스트

while True:
    # 검색 파라미터 설정
    query_params = {
        "query": urllib.parse.quote(searchWord),
        "display": display, 
        "start": start,
        "sort": sort
    }

    query_string = "&".join([f"{key}={value}" for key, value in query_params.items()])
    url = f"{base_url}?{query_string}"

    # API 요청 URL 출력
    print("요청 URL:", url)

    # 요청 객체 생성
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    # API 요청 및 응답 처리 
    try:
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200): # 성공 응답 
            response_body = response.read().decode('utf-8')
            data = json.loads(response_body)
            
            # 전체 결과 수와 현재 페이지의 아이템 가져오기
            #total_results = data["total"]
            total_results = 200 # test
            items = data["items"]
            all_items.extend(items)
            
            print(f"{start}번째 결과부터 {start + len(items) - 1}까지 데이터를 가져왔습니다.")
            
            # 다음 페이지로 이동
            start += display
            
            # 더 이상 가져올 데이터가 없으면 중단
            if start > total_results or not items:
                break
        else:
            print(f"Error Code: {response.getcode()}")
            break
    except Exception as e:
        print("Exception:", e)
        
# 모든 데이터 CSV로 저장
save_to_csv(all_items)

요청 URL: https://openapi.naver.com/v1/search/news.json?query=%EC%9E%AC%ED%85%8C%ED%81%AC&display=100&start=1&sort=sim
1번째 결과부터 100까지 데이터를 가져왔습니다.
요청 URL: https://openapi.naver.com/v1/search/news.json?query=%EC%9E%AC%ED%85%8C%ED%81%AC&display=100&start=101&sort=sim
101번째 결과부터 200까지 데이터를 가져왔습니다.
결과가 news_results.csv에 저장되었습니다.


In [76]:
import requests
from bs4 import BeautifulSoup
 
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}
data = requests.get('https://n.news.naver.com/mnews/article/081/0003510900?sid=101',headers=headers)
 
soup = BeautifulSoup(data.text, 'html.parser')

content = soup.select('#dic_area')
newspaper_name = soup.select_one('.media_end_head_top_logo_img').get("alt")
reporter_name = soup.select_one('.media_journalistcard_summary_name_text').text
subscribe_tag = soup.select('.media_journalistcard_summary_subscribe_value._SUBSCRIBE_COUNT_TEXT')

print(f"뉴스 내용 : \n{content}\n")
print(f"신문사 이름 : {newspaper_name}")
print(f"기자 이름 : {reporter_name}")
print(f"기자의 구독자 수 : {subscribe_tag[0]}")

뉴스 내용 : 
[<article class="go_trans _article_content" id="dic_area">
<strong class="media_end_summary">엔비디아에 HBM 공급 시 실적 개선 기대</strong><span class="end_photo_org"><div class="nbd_im_w _LAZY_LOADING_WRAP">
<div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
<img alt="서울 서초구 삼성전자 사옥. 2024.10.31 연합뉴스" class="_LAZY_LOADING _LAZY_LOADING_INIT_HIDE" data-src="https://imgnews.pstatic.net/image/081/2025/01/14/0003510900_001_20250114164822336.jpg?type=w860" id="img1" style="display: none;">
</img></div>
</div><em class="img_desc">서울 서초구 삼성전자 사옥. 2024.10.31 연합뉴스</em></span><br/><br/>지난해 삼성전자의 주가가 폭락하며 시가총액이 156조원 증발한 가운데, 올해 부활에 성공할지가 엔비디아에 달려있다는 전망이 나오고 있습니다. 삼성전자가 엔비디아 요구 사항을 충족시켜 고대역폭메모리(HBM) 공급에 성공할 경우 실적이 개선되겠지만, 이 과정이 순탄치만은 않을 것으로 예상되는 만큼 삼성전자의 기술력과 대응 능력이 시험대에 올랐다는 평가입니다.<br/><br/>13일(현지시간) CNBC 보도에 따르면, 삼성전자는 현재 엔비디아가 자사 고대역폭메모리(HBM) 사용을 승인하길 기다리고 있으며, 이는 회사 수익성 개선의 핵심 요소라는 분석이 나오고 있습니다.<br/><br/><!-- MobileAdNew center -->지난해 삼성전자 주가는 8만 7000원대까지 치솟았다가 연말까지 40%가량 떨어져 5만 3000원대로 주저앉았